In [6]:
# ==============================================================================
# Notebook 4 (v2): Simulação de Novos Clientes, Comparação e Conclusão
# ==============================================================================

# ------------------------------------------------------------------------------
# 1. Configuração e Carregamento de Bibliotecas e Artefatos
# ------------------------------------------------------------------------------
import pandas as pd
import numpy as np
import joblib
import os
import json
import shap
import matplotlib.pyplot as plt

In [7]:
# --- Definindo os caminhos conforme a estrutura de diretórios ---
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))

FINAL_MODEL_V1_PATH = os.path.join(ROOT_DIR, 'models', 'final_model', 'random_forest_final.pkl')
FINAL_MODEL_V2_PATH = os.path.join(ROOT_DIR, 'models', 'final_model_v2', 'random_forest_final_v2.pkl')
PREPROCESSING_PATH = os.path.join(ROOT_DIR, 'models', 'preprocessing')
REPORTS_PATH = os.path.join(ROOT_DIR, 'reports')
REPORTS_V2_PATH = os.path.join(ROOT_DIR, 'reports', 'v2')
PLOTS_V2_PATH = os.path.join(ROOT_DIR, 'reports', 'v2', 'plots')
X_TRAIN_PATH = os.path.join(ROOT_DIR, 'data', 'processed', 'split', 'X_train.csv')

In [8]:
# Carregando artefatos da versão 1
try:
    final_model_v1 = joblib.load(FINAL_MODEL_V1_PATH)
    metrics_v1_path = os.path.join(REPORTS_PATH, 'final_model_metrics.json')
    with open(metrics_v1_path, 'r') as f:
        metrics_v1 = json.load(f)
    print("✅ Artefatos da versão 1 carregados.")
except FileNotFoundError as e:
    print(f"❌ Erro ao carregar artefatos da v1: {e}. Certifique-se de que o notebook 3 original foi executado.")
    final_model_v1, metrics_v1 = None, None

# Carregando artefatos da versão 2
try:
    final_model_v2 = joblib.load(FINAL_MODEL_V2_PATH)
    scaler = joblib.load(os.path.join(PREPROCESSING_PATH, 'standard_scaler.pkl'))
    label_encoder_churn = joblib.load(os.path.join(PREPROCESSING_PATH, 'label_encoder_churn.pkl'))
    explainer_v2 = joblib.load(os.path.join(FINAL_MODEL_V2_PATH, 'shap_explainer_v2.pkl'))
    metrics_v2_path = os.path.join(REPORTS_V2_PATH, 'final_model_metrics_v2.json')
    with open(metrics_v2_path, 'r') as f:
        metrics_v2 = json.load(f)
    X_train = pd.read_csv(X_TRAIN_PATH)
    print("✅ Artefatos da versão 2 e de pré-processamento carregados.")
except FileNotFoundError as e:
    print(f"❌ Erro ao carregar artefatos da v2: {e}. Certifique-se de que o novo notebook 3 foi executado.")
    final_model_v2, scaler, label_encoder_churn, explainer_v2, metrics_v2, X_train = [None] * 6

✅ Artefatos da versão 1 carregados.
❌ Erro ao carregar artefatos da v2: [Errno 2] No such file or directory: 'c:\\temp\\Telecom_X_PrevendoChurn\\models\\final_model_v2\\random_forest_final_v2.pkl'. Certifique-se de que o novo notebook 3 foi executado.


In [9]:
if final_model_v2 is not None:
    # ------------------------------------------------------------------------------
    # 2. Simulação de Novos Dados
    # ------------------------------------------------------------------------------
    print("\nSimulando novos dados de clientes e fazendo previsões com a versão 2...")
    
    new_customers_data = {
        'gender': ['Female', 'Male'],
        'SeniorCitizen': [0, 1],
        'Partner': ['Yes', 'No'],
        'Dependents': ['Yes', 'No'],
        'tenure': [12, 1],
        'PhoneService': ['Yes', 'Yes'],
        'MultipleLines': ['Yes', 'No'],
        'InternetService': ['Fiber optic', 'DSL'],
        'OnlineSecurity': ['No', 'No'],
        'OnlineBackup': ['Yes', 'No'],
        'DeviceProtection': ['No', 'No'],
        'TechSupport': ['No', 'No'],
        'StreamingTV': ['Yes', 'No'],
        'StreamingMovies': ['Yes', 'No'],
        'Contract': ['Month-to-month', 'Month-to-month'],
        'PaperlessBilling': ['Yes', 'No'],
        'PaymentMethod': ['Electronic check', 'Mailed check'],
        'MonthlyCharges': [89.0, 45.0],
        'TotalCharges': [1068.0, 45.0]
    }
    
    new_customers_df = pd.DataFrame(new_customers_data)

In [10]:
# Pré-processamento
new_customers_df['gender'] = new_customers_df['gender'].map({'Female': 0, 'Male': 1})
new_customers_df['Partner'] = new_customers_df['Partner'].map({'No': 0, 'Yes': 1})
new_customers_df['Dependents'] = new_customers_df['Dependents'].map({'No': 0, 'Yes': 1})
new_customers_df['PhoneService'] = new_customers_df['PhoneService'].map({'No': 0, 'Yes': 1})
new_customers_df['PaperlessBilling'] = new_customers_df['PaperlessBilling'].map({'No': 0, 'Yes': 1})
    
new_customers_df = pd.get_dummies(new_customers_df, columns=[
    'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
    'PaymentMethod', 'InternetService'
], drop_first=True)
    
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
new_customers_df[numeric_cols] = scaler.transform(new_customers_df[numeric_cols])
    
new_customers_aligned = new_customers_df.reindex(columns=X_train.columns, fill_value=0)

NameError: name 'new_customers_df' is not defined

In [ ]:
# Previsão
churn_probabilities = final_model_v2.predict_proba(new_customers_aligned)[:, 1]
optimized_threshold = metrics_v2['best_threshold']
churn_predictions = (churn_probabilities >= optimized_threshold).astype(int)

new_customers_df['Probability of Churn'] = churn_probabilities
new_customers_df['Predicted Churn'] = label_encoder_churn.inverse_transform(churn_predictions)
    
print("✅ Previsões realizadas com sucesso com a versão 2.")
print("\nResultados da Simulação:")
print(new_customers_df[['Probability of Churn', 'Predicted Churn']])

In [ ]:
# ------------------------------------------------------------------------------
# 3. Análise SHAP para a Simulação
# ------------------------------------------------------------------------------
print("\n--- Análise de Interpretabilidade (SHAP) para os Clientes Simulados ---")
    
simulated_shap_values = explainer_v2.shap_values(new_customers_aligned)
    
for i, row in new_customers_aligned.iterrows():
    print(f"\nExplicação da Previsão para o Cliente #{i+1}:")
    shap.initjs()
    # Visualização com SHAP para a classe 'Churn' (classe 1)
    display(shap.force_plot(explainer_v2.expected_value[1], simulated_shap_values[1][i], row, show=False, matplotlib=True))
    plt.tight_layout()
    plt.savefig(os.path.join(PLOTS_V2_PATH, f'shap_force_plot_cliente_{i+1}.png'))
    plt.show()

In [ ]:
# ------------------------------------------------------------------------------
# 4. Comparação de Resultados e Conclusão
# ------------------------------------------------------------------------------
print("\n--- Conclusão do Projeto (Comparação V1 vs. V2) ---")
    
if final_model_v1 is not None:
    comparison_metrics = pd.DataFrame({
        'Métrica': ['Precisão', 'Recall', 'F1-Score', 'AUC'],
        'Modelo Otimizado (V1)': [
            metrics_v1['precision'],
            metrics_v1['recall'],
            metrics_v1['f1-score'],
            metrics_v1['auc_score']
        ],
        'Modelo Otimizado (V2)': [
            metrics_v2['precision'],
            metrics_v2['recall'],
            metrics_v2['f1-score'],
            metrics_v2['auc_score']
        ]
    })
    print("Comparação de Métricas de Desempenho:")
    print(comparison_metrics)
    
print("\nProcesso do Notebook 4 (v2) concluído com sucesso!")